## Convert Sentinel-2 image bands to GeoTIFF

First perform the three import statements that are required.

In [1]:
import xml.etree.ElementTree as xml
from osgeo import gdal
import os

Compute the full path to the unzipped Sentinel-2 folder: the .SAFE folder is the one required here.

In [2]:
homedir = '/home/eoafrica'
sentinelfile = '/eodata/Sentinel-2/MSI/L2A/2021/12/29/S2B_MSIL2A_20211229T074229_N0301_R092_T37PEK_20211229T101323.SAFE'
#S2B_MSIL2A_20220217T073949_N0400_R092_T37PEK_20220217T102748.SAFE
sentinelfile = homedir + sentinelfile

Get the list of band-filenames (10m, 20m, 60m, and also the additional bands like TCI and SCL) from the Sentinel-2's xml file (MTD_MSIL2A.xml) 

In [3]:
msil = open(sentinelfile + '/MTD_MSIL2A.xml')
msil = xml.parse(msil).getroot()
namespace_msil = msil.tag.split('}')[0] + '}'
filenames = msil.findall(namespace_msil + 'General_Info/Product_Info/Product_Organisation/Granule_List/Granule/IMAGE_FILE')
print('There are ' + str(len(filenames)) + ' items in the sentinel image')

There are 35 items in the sentinel image


Loop through the band-filenames and store them one by one as a GeoTIFF file in the same folder as this python notebook.

In [5]:
sequence = 1
for name in filenames:
    name = name.text + '.jp2'
    print('['+str(sequence)+'/'+str(len(filenames))+'] '+ name)
    outputname = name[0:-4] + '.tif'
    outputname = outputname.split('/')[-1]
    if os.path.exists(outputname):
        print('File ' + outputname + ' already exists; skipping')
        sequence = sequence + 1
        continue
    try:
        #dataset=gdal.Open(sentinelfile + '/' + name) # this does not work currently; the following four lines are the workaround
        f = open(sentinelfile + '/' + name, 'rb')
        mmap_name = "/vsimem/"+name
        gdal.FileFromMemBuffer(mmap_name, f.read())
        dataset=gdal.Open(mmap_name)
        dataset = gdal.Translate(outputname, dataset, format='GTiff')
        if dataset is None:
            print('Store failed')
        else:
            print('Stored as: ' + outputname)
        gdal.Unlink(mmap_name)
    except:
        print('no permission to read image')
    sequence = sequence + 1


[1/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_B02_10m.jp2
File T37PEK_20211229T074229_B02_10m.tif already exists; skipping
[2/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_B03_10m.jp2
File T37PEK_20211229T074229_B03_10m.tif already exists; skipping
[3/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_B04_10m.jp2
File T37PEK_20211229T074229_B04_10m.tif already exists; skipping
[4/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_B08_10m.jp2
Stored as: T37PEK_20211229T074229_B08_10m.tif
[5/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_TCI_10m.jp2
Stored as: T37PEK_20211229T074229_TCI_10m.tif
[6/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20211229T074229_AOT_10m.jp2
Stored as: T37PEK_20211229T074229_AOT_10m.tif
[7/35] GRANULE/L2A_T37PEK_A025143_20211229T075506/IMG_DATA/R10m/T37PEK_20

In [5]:
import shutil

In [6]:
destination_dir = '/home/eoafrica/EOAfrica_Awash_basin_project/Data/S2A_metadata'
shutil.copytree(sentinelfile, destination_dir)
print('Copied')

Copied


In [8]:
from osgeo import gdal

# Replace 'path_to_your_image.tif' with the actual path to your TIFF image
#Sentinel2_AWbasin/T37PEK_20211229T074229_B04_10m.tif
image_path = '/home/eoafrica/RF_model/T37PEK_20211229T074229_B04_10m.tif'

# Open the TIFF file
dataset = gdal.Open(image_path)

# Check if the dataset was successfully opened
if dataset is not None:
    print("Dataset opened successfully.")
    
    # Get image dimensions
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    bands = dataset.RasterCount
    print(f"Dimensions: {width} x {height} pixels, {bands} bands.")
    
    # Read the image data as a numpy array
    image = dataset.ReadAsArray()

    # Now, 'image' contains the pixel values of the TIFF image
    
    # Close the dataset
    dataset = None
else:
    print("Failed to open the dataset.")


Dataset opened successfully.
Dimensions: 10980 x 10980 pixels, 1 bands.


### Load the individual bands

In [19]:

##band_path = "path/to/your/bands"  # Replace with actual file paths
#band_path = '/home/eoafrica/Sentinel2_AWbasin'
#bands = [rasterio.open(f"{band_path}/B{i}.tif") for i in range(4, 4)]

In [ ]:
import rasterio

# Define paths to your Sentinel-2 band files
##band_path = "path/to/your/bands"  # Replace with actual file paths
band2_path = '/home/eoafrica/Sentinel2_AWbasin/B2.tif'
band3_path = '/home/eoafrica/Sentinel2_AWbasin/B3.tif'
band4_path = '/home/eoafrica/Sentinel2_AWbasin/B4.tif'
band8_path = '/home/eoafrica/Sentinel2_AWbasin/B8.tif' 
# Open the bands
with rasterio.open(band8_path) as src8, rasterio.open(band4_path)as src4, rasterio.open(band3_path) as src3: #, rasterio.open(band2_path) as src2:
    band8 = src8.read(1)  # Read band 8 data
    band4 = src4.read(1)  # Read band 4 data
    band3 = src3.read(1)  # Read band 3 data
    #band2 = src2.read(1)  # Read band 2 data